In [1]:
import cv2
import time

In [2]:
video=cv2.VideoCapture('Traffic_Laramie_1.mp4')
video2= cv2.VideoCapture('Traffic_Laramie_2.mp4')

In [3]:
# Check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")
    
#detect objects that are being captured
detect_objs = cv2.createBackgroundSubtractorMOG2(varThreshold=20,detectShadows=False)

#list to store the number of detected cars
detected_cars = []
car_counter = 0
save_prev_frame=0


In [4]:
frames_count, fps, width, height = video.get(cv2.CAP_PROP_FRAME_COUNT), video.get(cv2.CAP_PROP_FPS), video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)

width = int(width)
height = int(height)
print(f'Total frames: {frames_count}')
print(f'Fps: {fps}')
print(f'Video Width: {width}')
print(f'Video Height: {height}')

Total frames: 4448.0
Fps: 25.0
Video Width: 1040
Video Height: 600


The centroid is the centre of an object or figure. We compute the centre of each car rectangle, to determine if it has crossed a certain position in the frame, if it has, we count it as a car has been passed. Formula to calculate the centroid of a rectangle[1].

In [5]:
def get_centroid(x, y, w, h):
    cx = x + int(w/2)
    cy = y + int(h/2)
    return cx,cy

In [6]:
#read until video is completed or we press 'Q'
def analyze_video(video):
        # Check if the video opened successfully
    if (video.isOpened()== False): 
        print("Error opening video file")

    #detect objects that are being captured
    detect_objs = cv2.createBackgroundSubtractorMOG2(varThreshold=20,detectShadows=False)

    #list to store the number of detected cars
    detected_cars = []
    car_counter = 0
    save_prev_frame=0
    
    while True:    
        #capture frame-by-frame
        check, frame = video.read()
        current_frame = video.get(cv2.CAP_PROP_POS_FRAMES)
        #if we reach the end of the video, check will be false
        if not check:
                break

        #define 'main street''s dimension
        main_street = frame[300:430,1:1040]

        #apply gray and blur to region of interest, main_street
        gray = cv2.cvtColor(main_street,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5,5), 0 )

        foreground_mask  = detect_objs.apply(blur)


        #to apply to the morphology
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  
        closing = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening,kernel,iterations=2)



        #get contours from the foreground_mask
        contours,hier = cv2.findContours(foreground_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #rgb colors values
        green = (0,255,0)
        red = (0,0,255)

        #checkpoint 1
        cv2.line(main_street,(205,40),(205,145),green,2)


        for x in contours:
            contour_area = cv2.contourArea(x)
            if(contour_area>3000):
                #draw box for detected contours
                x,y,width,height = cv2.boundingRect(x)
                cv2.rectangle(main_street,(x,y),(x+width,y+height),green,2)

                #draw circle to track centroid of car
                car_centroid = get_centroid(x,y,width,height)
                get_cx,get_cy = get_centroid(x,y,width,height)

                cv2.circle(main_street,car_centroid,5,red,-1)


                 #testing for video 1 & 2, slow cars
                if(195 <= get_cx <= 205) and (80<= get_cy <=100):
                    #add coordinates to detected list
                    detected_cars.append(car_centroid)
                    for (cx,cy) in detected_cars:
                        if cx <(205+5) and cx > (205-4):
                            car_counter = car_counter + 1
                            detected_cars.remove((cx,cy))

        if current_frame == 1500:
            save_prev_frame = current_frame
            print('initial 1 min')
            print(car_counter)
        if current_frame == (save_prev_frame * 2):
            print('1 min has passed')
            print(car_counter)
            save_prev_frame = current_frame


        if check == True:
            cv2.imshow("Main_street_1.2",frame)
            #cv2.imshow("Frame Differencing",dilation)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    #after the loop release the video object
    video.release()
    #destroy all the windows
    cv2.destroyAllWindows()
    print(f'Total number of cars: {car_counter}')    

In [7]:
analyze_video(video)
analyze_video(video2)

initial 1 min
1
1 min has passed
3
Total number of cars: 5
initial 1 min
2
Total number of cars: 4


Cannot detect second car in laramie 1.


The range and off set value conditional sometimes is too tight, to detect the second car, but if the offset value and/or range detect increases, it double counts. Consider using euclidean distance to measure the car to the checkpoint.

# References

[1] www.engineeringtoolbox.com. (n.d.). Centroids of Plane Areas. [online] Available at: https://www.engineeringtoolbox.com/centroids-areas-d_2174.html.